In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, f1_score, accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, auc, roc_curve
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor  
import warnings
import pickle
import os
import json

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [2]:
out_dir = f'/home/meryem.janati/lustre/nlp_team-um6p-st-sccs-id7fz1zvotk/IDS/janati/IDS/Datasets/cicids17/Argus/default/CIC-IDS-2017_argus_default.csv'
df = pd.read_csv(out_dir)

In [7]:
df.columns

Index(['SrcId', 'Rank', 'StartTime', 'LastTime', 'Trans', 'Flgs', 'Seq', 'Dur',
       'RunTime', 'IdleTime',
       ...
       'dVpri', 'SRange', 'ERange', 'SrcTCPBase', 'DstTCPBase', 'TcpRtt',
       'SynAck', 'AckDat', 'TcpOpt', 'Attack'],
      dtype='object', length=109)

In [2]:
def save_scores(timeout, meanScores, stdScore):
    results = {
        'Timeout': timeout,
        'Mean of all scores': meanScores,
        'Std of all Scores': stdScores

    }

    with open(f'../Checkpoints/MLP/MLP_cic17_argus_{timeout}.json', 'w') as f:
        json.dump(results, f, indent=4)

In [3]:
# Features from paper 1 and paper 2
cols = ["Dur", "RunTime", "IdleTime", "Mean", "StdDev", "Sum", "Min", "Max", "Proto", "Cause", "TotPkts", "SrcPkts", "DstPkts", "TotBytes", "SrcBytes",
         "DstBytes", "Load", "SrcLoad", "DstLoad", "Rate", "SrcRate", "DstRate", "Attack",
        "SAppBytes", "DAppBytes", "SynAck", "AckDat", "TcpRtt"
        ]

len(cols)

28

In [4]:
timeouts = ['default', 0.5, 1, 2, 3, 4, 5, 6, 10, 30, 60]

### Training

In [5]:
best_f1 = 0
worst_f1 = 1
best_mean, worst_mean, best_std, worst_std = None, None, None, None

save=True

for timeout in timeouts:
    print("Processing timeout : ", timeout)
    
    if timeout =='default':
        out_dir = f'/home/meryem.janati/lustre/nlp_team-um6p-st-sccs-id7fz1zvotk/IDS/janati/IDS/Datasets/cicids17/Argus/{timeout}/CIC-IDS-2017_argus_{timeout}.csv'
    else:
        out_dir = f'/home/meryem.janati/lustre/nlp_team-um6p-st-sccs-id7fz1zvotk/IDS/janati/IDS/Datasets/cicids17/Argus/timeout{timeout}/CIC-IDS-2017_argus_{timeout}.csv'

    df = pd.read_csv(out_dir)
    df = df[cols]
    
    #df = df.drop(columns=['SrcId', 'SrcAddr', 'DstAddr', 'Sport', 'Dport', 'StartTime', 'LastTime', 'Dir', 'State', 'sIpId', 'sDSb', 'AutoId'])
    df = df.dropna() 
    # Convert categorical variables to numerical using Label Encoding
    label_encoders = {}
    for column in ['Proto', 'Cause', 'Flgs' ]:
        if column in df.columns:
            le = LabelEncoder()
            df[column] = le.fit_transform(df[column])
            label_encoders[column] = le

    # Split df into features and labels
    X = df.drop(columns=['Attack'])  # Assuming 'label' is the target variable
    y = df['Attack']
    
    
    accuracy, f1, precision, recall =[], [], [], []
    skf= StratifiedKFold(n_splits=5,random_state=None)
    skf.get_n_splits(X,y)
    
    for (train_index, test_index), i in zip(skf.split(X, y), range(5)):
        X_train,X_test=X.iloc[train_index],X.iloc[test_index]
        y_train,y_test=y.iloc[train_index],y.iloc[test_index]


        le = LabelEncoder()
        y_train = le.fit_transform(y_train)
        
        
        # Map unseen values to '<unknown>'
        y_test = y_test.map(lambda s: '<unknown>' if s not in le.classes_ else s)

        # Add '<unknown>' to classes and transform
        le.classes_ = np.append(le.classes_, '<unknown>')
        y_test = le.transform(y_test)
        
        
        
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        # Initialize and train Extra Trees Classifier
        clf = MLPClassifier(random_state=123, solver='adam', max_iter=8000)
        clf.fit(X_train, y_train)
        pred = clf.predict(X_test)
        
        f1Score = f1_score(y_true=y_test, y_pred=pred, average='macro')
        accScore=accuracy_score(y_test, pred)
        precScore = precision_score(y_test, pred, average='macro')
        recScrore = recall_score(y_test, pred, average='macro')
                             
        f1.append(f1Score)
        accuracy.append(accScore)
        precision.append(precScore)
        recall.append(recScrore)
        print('Fold: ', i, 'done!')
        
    meanScores, stdScores = {}, {}
    
    meanScores['f1Mean'] = np.array(f1).mean()
    meanScores['accMean'] = np.array(accuracy).mean()
    meanScores['recMean'] = np.array(recall).mean()
    meanScores['precMean'] = np.array(precision).mean()
    
    stdScores['f1Std'] = np.array(f1).std()
    stdScores['accStd'] = np.array(accuracy).std()
    stdScores['recStd'] = np.array(recall).std()
    stdScores['precStd'] = np.array(precision).std()
    
    print("Mean of all scores: ", meanScores)
    print("Std of all scores: ", stdScores)


    if save:
        save_scores(timeout, meanScores, stdScores)

    if meanScores['f1Mean'] > best_f1: 
        best_timeout = timeout
        best_mean = meanScores
        best_std = stdScores
        best_f1 = meanScores['f1Mean']
    
    if meanScores['f1Mean'] <= worst_f1: 
        
        worst_timeout = timeout
        worst_mean = meanScores
        worst_std = stdScores
        worst_f1 = meanScores['f1Mean']
               
    print('_______________________________________________')

Processing timeout :  default
Fold:  0 done!
Fold:  1 done!
Fold:  2 done!
Fold:  3 done!
Fold:  4 done!
Mean of all scores:  {'f1Mean': 0.6891242179756331, 'accMean': 0.9417454452755856, 'recMean': 0.6932933601823696, 'precMean': 0.7456622329314995}
Std of all scores:  {'f1Std': 0.026529470167881607, 'accStd': 0.005332333906523438, 'recStd': 0.03224483497342813, 'precStd': 0.008448639528230556}
_______________________________________________
Processing timeout :  0.5
Fold:  0 done!
Fold:  1 done!
Fold:  2 done!
Fold:  3 done!
Fold:  4 done!
Mean of all scores:  {'f1Mean': 0.6767232809467544, 'accMean': 0.9051536340097094, 'recMean': 0.7015667693979732, 'precMean': 0.7492361396108095}
Std of all scores:  {'f1Std': 0.0527324891320327, 'accStd': 0.07705991477260635, 'recStd': 0.02357261298697192, 'precStd': 0.06321749207359992}
_______________________________________________
Processing timeout :  1
Fold:  0 done!
Fold:  1 done!
Fold:  2 done!
Fold:  3 done!
Fold:  4 done!
Mean of all sco

In [2]:
def load_score(timeout):
    with open(f'../Checkpoints/MLP/MLP_cic17_argus_{timeout}.json', 'r') as f:
        loaded_results = json.load(f)
    return loaded_results


In [3]:
timeouts = ['default', 0.5, 1, 2, 3, 4, 5, 6, 10, 30, 60]
best_f1 = 0
worst_f1 = 1



for timeout in timeouts:
    loaded_results = load_score(timeout)
    print(loaded_results)
        
        
    if loaded_results['Mean of all scores']['f1Mean'] > best_f1: 
        best_timeout = loaded_results['Timeout']
        best_mean = loaded_results['Mean of all scores']
        best_std = loaded_results['Std of all Scores']
        best_f1 = loaded_results['Mean of all scores']['f1Mean'] 
    
    if loaded_results['Mean of all scores']['f1Mean'] <= worst_f1: 
        
        worst_timeout = loaded_results['Timeout']
        worst_mean = loaded_results['Mean of all scores']
        worst_std = loaded_results['Std of all Scores']
        worst_f1 = loaded_results['Mean of all scores']['f1Mean'] 
               
    

{'Timeout': 'default', 'Mean of all scores': {'f1Mean': 0.6891242179756331, 'accMean': 0.9417454452755856, 'recMean': 0.6932933601823696, 'precMean': 0.7456622329314995}, 'Std of all Scores': {'f1Std': 0.026529470167881607, 'accStd': 0.005332333906523438, 'recStd': 0.03224483497342813, 'precStd': 0.008448639528230556}}
{'Timeout': 0.5, 'Mean of all scores': {'f1Mean': 0.6767232809467544, 'accMean': 0.9051536340097094, 'recMean': 0.7015667693979732, 'precMean': 0.7492361396108095}, 'Std of all Scores': {'f1Std': 0.0527324891320327, 'accStd': 0.07705991477260635, 'recStd': 0.02357261298697192, 'precStd': 0.06321749207359992}}
{'Timeout': 1, 'Mean of all scores': {'f1Mean': 0.6864747015141759, 'accMean': 0.9384338873937471, 'recMean': 0.6943054877283292, 'precMean': 0.7231085000092445}, 'Std of all Scores': {'f1Std': 0.02694108060109457, 'accStd': 0.0031071041062293698, 'recStd': 0.02465865094676195, 'precStd': 0.029626897477333474}}
{'Timeout': 2, 'Mean of all scores': {'f1Mean': 0.65936

In [4]:
print("Best Timeout Combination: ", best_timeout)
print("Mean Scores (Best): ", best_mean)
print('Std Scores (Best):', best_std)

Best Timeout Combination:  6
Mean Scores (Best):  {'f1Mean': 0.7058858757605757, 'accMean': 0.9396982448994795, 'recMean': 0.7068976294594156, 'precMean': 0.7529652554141195}
Std Scores (Best): {'f1Std': 0.017176103209796314, 'accStd': 0.0025940373557517818, 'recStd': 0.02541991044042555, 'precStd': 0.01629369710967945}


In [5]:
print("worst Timeout Combination: ", worst_timeout)
print("Mean Scores (Worst): ", worst_mean)
print('Std Scores (Worst):', worst_std)

worst Timeout Combination:  5
Mean Scores (Worst):  {'f1Mean': 0.6492088099486735, 'accMean': 0.8632980506112204, 'recMean': 0.6938922911019576, 'precMean': 0.6976737003211599}
Std Scores (Worst): {'f1Std': 0.1176403418665107, 'accStd': 0.16082203372713053, 'recStd': 0.05655736636073002, 'precStd': 0.0868637402078548}


In [6]:

results = {
    'Best score': {
        'Best Timeout': best_timeout,
        'Mean Scores (Best)': best_mean,
        'Std Scores (Best)': best_std,
    },
    
    'Worst score': {
        'Worst Timeout': worst_timeout,
        'Mean Scores (Worst)': worst_mean,
        'Std Scores (Worst)': worst_std,
    },
    
    'Difference': {
        'Accuracy': (best_mean['accMean'] - worst_mean['accMean'])*100,
        'F1 Score': (best_mean['f1Mean'] - worst_mean['f1Mean'])*100,
        'Precision': (best_mean['precMean'] - worst_mean['precMean'])*100,
        'Recall': (best_mean['recMean'] - worst_mean['recMean'])*100
    }
}



with open('../results/MLP_cic17_argus.json', 'w') as f:
    json.dump(results, f, indent=4)